## Notes 

- rens van der schoot
- 


## Ideas

- symantic web technology, micro applications (combine the approaches)
- the role of authors, citations
- sensitivity 1 doesn't work. (multiple raters, different expertise)
- citation networks
- use of rules how to interpreted the text. 
- 






## Remarks

- Covidence and Rayyan (software)
- 



# Systematic review

Based on: https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py

> This script loads pre-trained word embeddings (GloVe embeddings)
into a frozen Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classification of newsgroup messages into 20 different categories).
GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)
20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html


## Related links
https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/
https://machinelearningmastery.com/best-practices-document-classification-deep-learning/
https://www.quora.com/What-deep-learning-method-to-use-to-classify-text-files

Import dependencies

In [1]:
%matplotlib inline

In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
import pandas as pd
import sklearn
import keras_metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras import metrics
from keras.callbacks import TensorBoard

from nltk.corpus import stopwords
from prettytable import PrettyTable
from numpy import cumsum
from matplotlib import pyplot
from pandas import DataFrame
from datetime import datetime
from scipy import stats
from keras.layers import Bidirectional

Using TensorFlow backend.


## Project properties

Basic output properties related to the folder structure.

In [2]:
# OUTPUT_DIR = os.path.join("..", "output")

LOG_DIR = os.path.join("..","logs")

## Evaluation Criteria

- minimize number of papers
- max # of fn =1
- threshhold >0

## Data

Basic data related variables.

In [3]:
# the data folder
BASE_DIR = os.path.join("..", "data")

# the folder with the word2vec 
GLOVE_DIR = os.path.join("..", "word2vec")

# the papers about PTSD
TEXT_DATA_DIR = os.path.join(BASE_DIR, "ptsd_review", "csv")

# the target variable
TARGET_VARIABLE = "included_final" # "included_ats" (after title screening)


Read data in memory.

In [4]:
# second, prepare text samples and their labels
print('Processing text dataset')

data_path = os.path.join(TEXT_DATA_DIR, "schoot-lgmm-ptsd-traindata.csv")
full_data = pd.read_csv(data_path)

texts = (full_data['title'].fillna('') + ' ' + full_data['abstract'].fillna(''))
labels = full_data[TARGET_VARIABLE]
print('Found %s texts.' % len(texts))

Processing text dataset
Found 5077 texts.
<class 'pandas.core.series.Series'>


In [9]:
t=[0, 0, 0, 0, 17398, 18790, 19098, 1568, 8762, 10384, 8524, 15750, 22423, 8762, 10384, 10731, 11841, 16419, 7688, 6968, 22154, 11841, 1568, 3104, 18790, 18813, 2759, 15616, 16419, 1942, 5526, 16419, 7884, 15334, 13962, 12390, 16715, 3980, 18790, 19098, 1568, 824, 15026, 11841, 1568, 14202, 1411, 290, 23501, 6968, 15319, 20205, 18813, 13296, 17707, 16390, 14608, 18813, 3458, 2603, 18813, 19567, 6058, 18813, 22041, 18813, 6968, 24069, 15616, 24740, 18813, 11768, 16200, 16419, 824, 22733, 11841, 24943, 1568, 290, 3983, 6968, 24521, 18813, 15943, 18813, 5494, 18790, 18813, 6968, 3977, 5602, 16419, 17407, 12342, 11841, 24943, 1568, 290, 23501, 6833, 18813, 8188, 18813, 9341, 18813, 13229, 18813, 12819, 16878, 17563, 18523, 18813, 6968, 6899, 12390, 16419, 1905, 7372, 11841, 16419, 13612, 9829, 14202, 11161, 23962, 12390, 14416, 9881, 18813, 18343, 3974, 4472, 4182, 19525, 14222, 4834, 11701, 24355, 11669, 16419, 3458, 9896, 11841, 20205, 6968, 12213, 13316, 15341, 10351, 11841, 16419, 9168, 11841, 21219, 12390, 24796, 20510, 7668, 17655, 25333, 7289, 6968, 22653, 13316, 15341, 14241, 11841, 8524, 12390, 5720, 9752, 11841, 15950, 3245, 21836, 13322, 17399, 12390, 9314, 9150, 6968, 16419, 487, 24854, 9314, 21630, 16419, 7983, 21219, 11841, 1568, 3104, 18790, 8762, 10384, 3020, 4977, 7394, 1568, 3104, 18790, 16419, 5644, 947, 13537, 4025, 2510, 11841, 16419, 5894, 11841, 16419, 21219, 18813, 16419, 20361, 19013, 13962, 12390, 23402, 11768, 18813, 6968, 21836, 7884, 23489, 2366, 5013, 16419, 17587, 13962, 12390, 24612, 18518, 15682, 14202, 1047, 23962, 14841, 17776, 20894, 12185, 17655, 9648, 290, 16419, 19026, 12916, 1385, 4977, 17284, 16419, 12536, 11841, 3126, 23543, 17655, 12530, 14973, 16419, 13322, 24854, 1662, 6968, 23829, 18790, 290, 16419, 9752, 11841, 10920, 15950, 24943, 5232, 22451, 447, 6968, 20648, 13240, 14648, 14841, 3245, 16419, 6249, 17655, 13962, 15616, 16419, 6903, 11841, 18790, 17655, 23962, 290, 24898, 12390, 16715, 3980, 18790, 19567, 9314, 20279, 23006, 23178, 19536, 17998, 13965, 14202, 1047, 1411, 12530, 14973, 22223, 17864, 18629, 2851, 18813, 6495, 18813, 17276, 18813, 2442, 18813, 3417, 6968, 1905, 22166, 13156, 17361, 24854, 17398, 18790, 19098, 1568, 6968, 20499, 8390, 4902, 14202, 1047, 8693, 16419, 5644, 7080, 13537, 3020, 4977, 12812, 13322, 8253, 14202, 19610, 22028, 19098, 4025, 13669, 18551, 11841, 51, 5232, 24943, 18813, 16419, 5644, 1721, 12390, 12019, 6968, 4941, 10239, 15616, 23325, 18813, 3293, 18813, 265, 18813, 2442, 18813, 1847, 18813, 6968, 1905, 14843, 18813, 5232, 15796, 5232, 14184, 18813, 1905, 22563, 18813, 25140, 19552, 18813, 22210, 18813, 6968, 4807, 15718, 10730, 14973, 14843, 21719, 11608, 21637, 22235, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
np.array(t,dtype='int32')

array([0, 0, 0, ..., 0, 0, 0])

## Word tokenizing

Apply vectorization to texts.

In [6]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000

In [7]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# get the word index
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='post', truncating='post')
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 26396 unique tokens.
Shape of data tensor: (5077, 1000)
Shape of label tensor: (5077, 2)


## Create training and validation set
Split the dataset into a training and a validation set. Use SciKit-learn for this. 
We decided to train our model on a small dataset (around 500), to make the environment more simulated to the active learning approach where we have small number of labeled data points. 

In [8]:
from sklearn.model_selection import train_test_split

def split_data(validation_split, added_positives):
    x_train, x_val, y_train, y_val = train_test_split(
        data,
        labels,
        test_size=validation_split,
        random_state=2018,
        stratify=labels
    )

    # add added_positives positive paper to training dataset
    if added_positives>0 :
        positive_indx = np.where(y_val[:,1]==1)[0]
        x_train = np.vstack((x_train, x_val[positive_indx[0:added_positives]]))
        y_train = np.vstack((y_train,y_val[positive_indx[0:added_positives]]))

        x_val = np.delete(x_val, positive_indx[0:added_positives],0)
        y_val = np.delete(y_val, positive_indx[0:added_positives],0)

    return (x_train, x_val, y_train, y_val)

x_train, x_val, y_train, y_val = split_data(0.2, 0)

print("x_train shape:", x_train.shape, ", x_val shape:", x_val.shape)
print("y_train shape:", y_train.shape, ", y_val shape:", y_val.shape)
print((y_train[:,1]==1).sum())
print((y_val[:,1]==1).sum())


x_train shape: (4061, 1000) , x_val shape: (1016, 1000)
y_train shape: (4061, 2) , y_val shape: (1016, 2)
32
8


We added 15 positive cases to training, it causes that model label all data as positive. Now I try with stratified samples.

## Prepare embedding layer


In [9]:
## set dimensions
EMBEDDING_DIM = 300

In [10]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'wiki.en.vec'), encoding='utf8') as f:
    for line in f:
        
        values = line.split()
        split_on_i = len(values) - EMBEDDING_DIM
        word = ' '.join(values[0:split_on_i])
        coefs = np.asarray(values[split_on_i:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 2518927 word vectors.


In [12]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
            continue
        
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
                
print('Shape of embedding matrix: ', embedding_matrix.shape)

Shape of embedding matrix:  (20000, 300)


## Make an Keras embedding layer.

In [13]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
def build_embedding(weights):
    return Embedding(num_words,
                                EMBEDDING_DIM,
                                weights=[weights],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)

embedding_layer = build_embedding(embedding_matrix)

## Models

This section contains models to use later on. Focusing on Convolutional and LSTM models. 

### LSTM

In [14]:
def get_lstm_model(backwards, dropout, neurons,optimizer ):
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    x = LSTM(neurons,input_shape=(MAX_SEQUENCE_LENGTH,),  go_backwards=backwards, dropout=dropout)(embedded_sequences)
    x = Dense(128,activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model_lstm = Model(inputs=sequence_input, outputs=output)

    model_lstm.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc',keras_metrics.recall()])

    model_lstm.summary()
    return model_lstm

def get_bi_lstm_model(mode, dropout, neurons, optimizer):
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    x = Bidirectional(LSTM(neurons,input_shape=(MAX_SEQUENCE_LENGTH,), dropout=dropout),merge_mode= mode)(embedded_sequences)
    x = Dense(128, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model_lstm = Model(inputs=sequence_input, outputs=output)

    model_lstm.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc',keras_metrics.recall()])

    model_lstm.summary()
    return model_lstm


### Convnet

In [ ]:
# # train a 1D convnet with global maxpooling
# def build_convnet(embedded_sequences):
#     x = Conv1D(128, 5, activation='relu')(embedded_sequences)
#     x = MaxPooling1D(5)(x)
#     x = Conv1D(128, 5, activation='relu')(x)
#     x = MaxPooling1D(5)(x)
#     x = Conv1D(128, 5, activation='relu')(x)
#     x = GlobalMaxPooling1D()(x)
#     x = Dense(128, activation='relu')(x)
#     preds = Dense(2, activation='softmax')(x)

#     model_convnet = Model(inputs=sequence_input, outputs=preds)

#     model_convnet.compile(loss='binary_crossentropy',
#                   optimizer='rmsprop',
#                   metrics=['acc',keras_metrics.recall()])
#     return model_convnet
    
    
# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# ##build convnet model
# embedded_sequences = embedding_layer(sequence_input)
# model_convnet = build_convnet(embedded_sequences)

## Train the model

### calculating class weights

In [15]:
weights = [1/y_val[:, 0].mean(), 1/y_val[:, 1].mean()]
weights

[1.0079365041220509, 127.00000047311187]

### Visualize model scores with TensorBoard

In [16]:
def save_run(sub_dir=None):
    
    if sub_dir is not None:
        log_dir = os.path.join(LOG_DIR, sub_dir)
    else:
        log_dir = LOG_DIR
    
    return TensorBoard(log_dir=log_dir, histogram_freq=0,
                       write_graph=True, write_images=True)
    


In [17]:
def train_model(model, epoch_no):
    
    # fit model for one epoch on this sequence
    start=datetime.now()
    hist = model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=epoch_no,
            validation_data=(x_val, y_val),
            shuffle=True,
            class_weight=weights,
            verbose = 0
        )  

    runtime=datetime.now() - start
    
    loss = hist.history['loss'][0]    
    return (model,loss, runtime)
 

def get_scores(model, threshhold):
    prediction = model.predict(x_val)
    y_classes = ( [0 if x < threshhold else 1 for x in prediction[:,1]])
    y_value = y_val.argmax(axis=1)
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_value,y_classes).ravel()
    return (tn, fp, fn, tp)


def get_pred(model):
    prediction = model.predict(x_val)
    return prediction


def get_scores_pred(pred, threshhold):
    y_classes = ( [0 if x < threshhold else 1 for x in pred[:,1]])
    y_value = y_val.argmax(axis=1)
    (tn, fp, fn, tp) = sklearn.metrics.confusion_matrix(y_value,y_classes).ravel()
    return (tn, fp, fn, tp)


### Compare Various LSTM models
Comparing LSTM forward, LSTM backward and Bi-directional LSTM

In [18]:
epochs = 5
BATCH_SIZE = 128
dropout=0
neurons=10
optimizer='rmsprop'

In [19]:
# lstm forwards
print('lstm forwards')
model = get_lstm_model(False,dropout,neurons,optimizer)
(trained_model,loss, runtime) = train_model(model, epochs)
forward_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

lstm forwards
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                12440     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1408      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 6,014,106
Trainable params: 14,106
Non-trainable params: 6,000,000
_________________________________________________________________
loss: 0.452999496285 runtime: 245.833196


In [36]:
print(stats.describe(forward_pred[:,1]))
threshhold=0.007
(tn, fp, fn, tp) = get_scores_pred(forward_pred, threshhold)
print(tn, fp, fn, tp)
forward_result = ('forward',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())

DescribeResult(nobs=1016, minmax=(0.0004685364, 0.0081466679), mean=0.0081391074, variance=5.8025279e-08, skewness=-31.827655792236328, kurtosis=1011.0000683884579)
1 1007 0 8


In [21]:
# lstm backwards
print('lstm_back')
model = get_lstm_model(True,dropout,neurons,optimizer)
(trained_model,loss, runtime) = train_model(model, epochs)
backward_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

lstm_back
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                12440     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1408      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258       
Total params: 6,014,106
Trainable params: 14,106
Non-trainable params: 6,000,000
_________________________________________________________________
loss: 0.19080518443 runtime: 423.034718


In [37]:
print(stats.describe(backward_pred[:,1]))
threshhold=0.008
(tn, fp, fn, tp) = get_scores_pred(backward_pred, threshhold)
print(tn, fp, fn, tp)
backward_result = ('backward',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())

DescribeResult(nobs=1016, minmax=(0.0010104601, 0.11843406), mean=0.0077425269, variance=8.161911e-05, skewness=6.076805591583252, kurtosis=55.52590563550463)
708 300 2 6


In [23]:
# bidirectional concat
print('bidirectional concat')
model = get_bi_lstm_model('concat', 0,neurons,optimizer)
(trained_model,loss, runtime) = train_model(model, epochs)
bilstm_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

bidirectional concat
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20)                24880     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               2688      
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 258       
Total params: 6,027,826
Trainable params: 27,826
Non-trainable params: 6,000,000
_________________________________________________________________
loss: 0.219225970372 runtime: 6463.14421


In [38]:
print(stats.describe(bilstm_pred[:,1]))
threshhold=0.008
(tn, fp, fn, tp) = get_scores_pred(bilstm_pred, threshhold)
print(tn, fp, fn, tp)
bi_result = ('bidirectional',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())

# # line plot of results
# results.plot()
# pyplot.show()

DescribeResult(nobs=1016, minmax=(0.0010974816, 0.086877428), mean=0.012040269, variance=6.1617146e-05, skewness=3.0470035076141357, kurtosis=17.544552364179676)
326 682 2 6
+---------------+------------+-----+------+----+----+----------------+---------------+
|     Model     | threshhold |  TN |  FP  | FN | TP |      Loss      | RunTime (sec) |
+---------------+------------+-----+------+----+----+----------------+---------------+
|    forward    |   0.007    |  1  | 1007 | 0  | 8  |     0.452      |    245.833    |
|    backward   |   0.008    | 708 | 300  | 2  | 6  |      0.19      |    423.034    |
| bidirectional |   0.008    | 326 | 682  | 2  | 6  | 0.219225970372 |   6463.14421  |
+---------------+------------+-----+------+----+----+----------------+---------------+


### Result
As it can be seen in the following table, for 5 epochs, without any dropout, backward LSTM achieve the best result. For each model, we found the bestt threshhold to meet our evaluation criteria.


In [ ]:
t = PrettyTable(['Model','threshhold','TN', 'FP', 'FN','TP','Loss','RunTime (sec)'])

t.add_row(forward_result)
t.add_row(backward_result)
t.add_row(bi_result)

print(t)

In [ ]:
# results = DataFrame()
# # sum merge
# model = get_bi_lstm_model(embedded_sequences, 'sum')
# results['bilstm_sum'] = train_model(model, 4)
# # mul merge
# model = get_bi_lstm_model(embedded_sequences, 'mul')
# results['bilstm_mul'] = train_model(model, 4)
# # avg merge
# model = get_bi_lstm_model(embedded_sequences, 'ave')
# results['bilstm_ave'] = train_model(model, 4)
# # concat merge
# model = get_bi_lstm_model(embedded_sequences, 'concat')
# results['bilstm_con'] = train_model(model, 4)
# # line plot of results
# results.plot()
# pyplot.show()

## Grid Search on hyperparameters

In [45]:
from datetime import datetime
from keras.layers import Bidirectional
from sklearn.model_selection import ParameterGrid


def get_threshholds(pred):
    desc= stats.describe(pred[:,1])
    (min_prob,max_prob) = desc.minmax
    step = (max_prob - min_prob)/10
    threshholds = ','.join(str(i) for i in range(min_prob,max_prob+step,step))
    return threshholds

def run_model(model_name,neurons,optimizer,epoch, dropout):
    
    all_scores = list()
    model = None
    if model_name =='bidirectional':
        print('lstm_bidirectional')
        model = get_bi_lstm_model('concat', dropout,neurons,optimizer)
    else:
        if model_name =='backward':
            print('lstm_backward')
            model = get_lstm_model(True,dropout,neurons,optimizer)

        
    (trained_model,loss, runtime) = train_model(model, epoch)
    pred = get_pred(trained_model)
    
    ## generate a sequence of 10 threshhold probabilities
    threshholds = get_threshholds(pred)
    
    ## calculate scores for each threshhold
    for threshhold in threshholds:
        scores = get_scores(trained_model, threshhold)
        scores_row = (model_name, neurons,optimizer,epoch, dropout,threshhold,scores,loss, runtime.total_seconds())
        all.scores.append(scores_row)
        
    return all_scores
      
     
        
        
def grid_search_lstm(validation_set, added_positives):        
    
    x_train, x_val, y_train, y_val = split_data(validation_set, added_positives)

    print("x_train shape:", x_train.shape, ", x_val shape:", x_val.shape)
    print("y_train shape:", y_train.shape, ", y_val shape:", y_val.shape)
    print("positive papers in train dataset:",(y_train[:,1]==1).sum())
    print("positive papers in test dataset:",(y_val[:,1]==1).sum())

    
    param_grid = {'model':['bidirectional','backward'], 'optimizer':['rmsprop','adam'],'neurons':[2,5,10], 
                  'epoch': [5, 10], 'dropout':[0,0.1,0.2,0.3,0.4,0.5,0.7]}
    grid = ParameterGrid(param_grid)
    
    t = PrettyTable(['Model', 'Neurons','Optimizer','Epoch','Dropout','Threshhold','TN', 'FP', 'FN', 'TP','Loss','RunTime (sec)'])
    
    
    results = list()
    for params in grid:
        
        print('model:',params['model'],'optimizer:',params['optimizer'], 'neurons:',params['neurons'],
              'epoch:',params['epoch'],' , dropout:',params['dropout'],)
        result = run_model(params['model'],params['neurons'],params['optimizer'], params['epoch'], params['dropout'])
        results.add(result)
        t.add_row(result)
        print(t) 
        
    return results  


In [46]:
## train model on 4000 datapoints
results4000 = grid_search_lstm(0.2, 0)

x_train shape: (4061, 1000) , x_val shape: (1016, 1000)
y_train shape: (4061, 2) , y_val shape: (1016, 2)
positive papers in train dataset: 32
positive papers in test dataset: 8
epoch: 1  , dropout: 0
lstm_bidirectional
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 20)                24880     
_________________________________________________________________
dense_13 (Dense)             (None, 128)               2688      
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 258       
Total params: 6,027,826
Trainable params: 27,826
Non-t

KeyboardInterrupt: 

In [ ]:
## train model on 500 datapoints
results500 = grid_search_lstm(0.9, 16)

In [ ]:
## train model on 100 datapoints
results100 = grid_search_lstm(0.98, 20)

In [ ]:
## train model on 50 datapoints
results50 = grid_search_lstm(0.99, 10)

In [ ]:
## train model on 20 datapoints
results20 = grid_search_lstm(0.996, 10)